In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import carla #the sim library itself
import time # to set a delay after each photo
import cv2 #to work with images from cameras
import numpy as np #in this example to change image representation - re-shaping
import math
import sys
import random
sys.path.append(r'C:\Users\Administrator\Downloads\WindowsNoEditor\PythonAPI\carla') # tweak to where you put carla

from agents.navigation.global_route_planner import GlobalRoutePlanner
from matplotlib import pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
import open3d as o3d

from PIL import Image

PREFERRED_SPEED = 20
SPEED_THRESHOLD = 2

MAX_STEER_DEGREES = 30
STEERING_CONVERSION = 75

CAMERA_POS_Z = 1.3
CAMERA_POS_X = 1.4

HEIGHT = 180
WIDTH = 320
X_MIN = -6
X_MAX = -1
Y_MIN = -1
Y_MAX = 1
Z_MIN = -1
Z_MAX = 2

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")


def update_spectator(vehicle):
    vehicle_transform = vehicle.get_transform()

    # Position the spectator slightly behind and above the vehicle
    spectator_location = vehicle_transform.location + carla.Location(x=0, z=40)  # Adjust x and z values as needed for view angle
    spectator_rotation = carla.Rotation(pitch=-90, yaw=vehicle_transform.rotation.yaw)  # Adjust pitch for angle

    spectator_transform = carla.Transform(spectator_location, spectator_rotation)
    spectator.set_transform(spectator_transform)

pcd1 = []

def lidar_callback(point_cloud, point_list):
    """Prepares a point cloud with intensity
    colors ready to be consumed by Open3D"""
    data = np.copy(np.frombuffer(point_cloud.raw_data, dtype=np.dtype('f4')))
    data = np.reshape(data, (int(data.shape[0] / 4), 4))
    points = data[:, :-1]
    points[:, :1] = -points[:, :1]

    pcd1.append(points)

def generate_lidar_bp(blueprint_library):
    """Generates a CARLA blue
    based on the script parameters"""
    lidar_bp = blueprint_library.find('sensor.lidar.ray_cast')

    lidar_bp.set_attribute('upper_fov', str(10.0))
    lidar_bp.set_attribute('lower_fov', str(-30.0))
    lidar_bp.set_attribute('channels', str(64.0))
    lidar_bp.set_attribute('range', str(100.0))
    lidar_bp.set_attribute('rotation_frequency', str(20))
    lidar_bp.set_attribute('points_per_second', str(1000000))
    return lidar_bp


def filter_points(points, x_min=X_MIN, x_max=X_MAX, y_min=Y_MIN, y_max=Y_MAX, z_min=Z_MIN, z_max=Z_MAX):
    mask = (
        (points[:, 0] >= x_min) & (points[:, 0] <= x_max) &  # x-axis (front)
        (points[:, 1] >= y_min) & (points[:, 1] <= y_max) &  # y-axis (lateral)
        (points[:, 2] >= z_min) & (points[:, 2] <= z_max)    # z-axis (height)
    )
    return np.any(mask), mask, points[mask]  # Return if any point is in front and the points themselves

def camera_callback(image,data_dict):
    data_dict['image'] = np.reshape(np.copy(image.raw_data),(image.height,image.width,4))[:, :, :3]


def sem_callback(image,data_dict):
    image.convert(carla.ColorConverter.CityScapesPalette)
    data_dict['sem_image'] = np.reshape(np.copy(image.raw_data),(image.height,image.width,4))[:, :, :3]


def maintain_speed(s):
    if s >= PREFERRED_SPEED:
        return 0
    elif s < PREFERRED_SPEED - SPEED_THRESHOLD:
        return 0.7 # think of it as % of "full gas"
    else:
        return 0.4 # tweak this if the car is way over or under preferred speed


# function to get angle between the car and target waypoint
def get_angle(car,wp):
    vehicle_pos = car.get_transform()
    car_x = vehicle_pos.location.x
    car_y = vehicle_pos.location.y
    wp_x = wp.transform.location.x
    wp_y = wp.transform.location.y

    # vector to waypoint
    x = (wp_x - car_x)/((wp_y - car_y)**2 + (wp_x - car_x)**2)**0.5
    y = (wp_y - car_y)/((wp_y - car_y)**2 + (wp_x - car_x)**2)**0.5

    #car vector
    car_vector = vehicle_pos.get_forward_vector()
    degrees = math.degrees(np.arctan2(y, x) - np.arctan2(car_vector.y, car_vector.x))
    # extra checks on predicted angle when values close to 360 degrees are returned
    if degrees<-180:
        degrees = degrees + 360
    elif degrees > 180:
        degrees = degrees - 360
    return degrees

def get_proper_angle(car,wp_idx,rte):
    # create a list of angles to next 5 waypoints starting with current
    next_angle_list = []
    for i in range(5):
        if wp_idx + i*3 <len(rte)-1:
            next_angle_list.append(get_angle(car,rte[wp_idx + i*3][0]))
    idx = 0
    while idx<len(next_angle_list)-2 and abs(next_angle_list[idx])>40:
        idx +=1
    return wp_idx+idx*3,next_angle_list[idx]

def get_distant_angle(car,wp_idx,rte, delta=10):
    if wp_idx + delta < len(rte)-1:
        i = wp_idx + delta
    else:
        i = len(rte)-1
    intersection_detected = False
    for x in range(i-wp_idx):
        if rte[wp_idx+x][0].is_junction:
             intersection_detected = True
    angle = get_angle(car,rte[i][0])
    if not intersection_detected:
        result = 1
    elif angle <-10:
        result = 0
    elif angle>10:
        result = 2
    else:
        result = 1
    return result

def draw_route(world, wp, route,seconds=3.0):
    if len(route)-wp <25:
        draw_colour = carla.Color(r=255, g=0, b=0)
    else:
        draw_colour = carla.Color(r=0, g=0, b=255)
    for i in range(10):
        if wp+i<len(route)-2:
            world.debug.draw_string(route[wp+i][0].transform.location, '^', draw_shadow=False,
                color=draw_colour, life_time=seconds,
                persistent_lines=True)
    return None

def select_random_route(world, position,locs):
    point_a = position.location #we start at where the car is or last waypoint
    sampling_resolution = 1
    grp = GlobalRoutePlanner(world.get_map(), sampling_resolution)
    # now let' pick the longest possible route
    min_distance = 100
    result_route = None
    route_list = []
    for loc in locs: # we start trying all spawn points
                                #but we just exclude first at zero index
        cur_route = grp.trace_route(point_a, loc.location)
        if len(cur_route) > min_distance:
            route_list.append(cur_route)
    result_route = random.choice(route_list)
    return result_route


def exit_clean(world, vehicle):
    for sensor in world.get_actors().filter('*sensor*'):
        sensor.destroy()
    # vehicle.destroy
    for actor in world.get_actors().filter('*vehicle*'):
        actor.destroy()
    return None

def collision_callback(event,data_dict):
    data_dict['collision']=True


def preprocessing_image(image):
    image = image.convert('L')
    image = image.point(lambda x: 255 if x >= 70 else 0, '1')
    width, height = image.size
    left = (width - 300) // 2
    top = (height - 250) // 2
    right = left + 300
    bottom = top + 250
    return image.crop((left, top, right, bottom))


class AutoNaviModel(nn.Module):
    def __init__(self):
        super(AutoNaviModel, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(64 * (HEIGHT // 8) * (WIDTH // 8), 128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, 4),
            nn.ReLU(),
            nn.Flatten()
        )
        self.output_layer = nn.Linear(5, 1)  # 4 from image + 1 from input_2

    def forward(self, image, navigational_direction):
        image_features = self.conv_layers(image)

        combined = torch.cat((image_features, navigational_direction.unsqueeze(1)), dim=1)

        output = self.output_layer(combined)
        return output


def control_car(sem_image, navigational_direction):
    sem_image_pil = Image.fromarray(sem_image.astype('uint8'), 'RGB')

    transform1 = transforms.Compose([
        transforms.Resize((HEIGHT, WIDTH)),
        transforms.ToTensor(),
    ])

    sem_image = transform1(sem_image_pil).unsqueeze(0).to(DEVICE)
    navigational_direction = torch.tensor([navigational_direction])

    with torch.no_grad():
        auto_navi_output = auto_navi_model(sem_image, navigational_direction)

    return float(auto_navi_output)


auto_navi_model_path = r'models\auto_navi_model.pth'
auto_navi_model = torch.load(auto_navi_model_path, map_location=torch.device(DEVICE))
auto_navi_model.eval()

#main loop
client = carla.Client('localhost', 2000)
world = client.get_world()
settings = world.get_settings()
settings.synchronous_mode = True  # Enable synchronous mode
settings.fixed_delta_seconds = 0.05  # Set the simulation time step (20 Hz)
world.apply_settings(settings)

spawn_points = world.get_map().get_spawn_points()

vehicle_bp = world.get_blueprint_library().filter('*model3*')
quit = False
WIDTH = 640
HEIGHT = 480


try:
    while True:
        print("-----------Start Iteration------------")

        vehi = 0
        # Create a traffic manager and set its parameters
        traffic_manager = client.get_trafficmanager(8000)  # You can specify a different port if needed
        traffic_manager.set_global_distance_to_leading_vehicle(2.0)  # Example setting: minimum distance to the leading vehicle
        traffic_manager.set_synchronous_mode(True)

        # Now integrate it with vehicle spawning
        for i in range(0, len(spawn_points), 6):
            start_point = spawn_points[i]
            vehicle_blueprints = world.get_blueprint_library().filter('vehicle.*')
            vehicle_bp = random.choice(vehicle_blueprints)
            vehicle_1 = world.try_spawn_actor(vehicle_bp, start_point)
            time.sleep(0.5)

            if vehicle_1:  # If the vehicle was successfully spawned
                traffic_manager.ignore_lights_percentage(vehicle_1, 0)  # Example setting: vehicles stop at traffic lights
                traffic_manager.vehicle_percentage_speed_difference(vehicle_1, -10)  # Reduce vehicle speed by 10%
                vehicle_1.set_autopilot(True, traffic_manager.get_port())  # Connect the vehicle with the traffic manager
                vehi += 1
                # print(vehi)
            # else:
            #     print(f"Failed to spawn vehicle at index {i}.")

        # start_point = random.choice(spawn_points)
        vehicle_bp = world.get_blueprint_library().filter('*model3*')
        start_point = spawn_points[2]
        # start_point =  random.choice(spawn_points)
        vehicle = world.try_spawn_actor(vehicle_bp[0], start_point)
        while vehicle is None:
            # start_point =  random.choice(spawn_points)
            start_point = spawn_points[2]
            vehicle = world.try_spawn_actor(vehicle_bp[0], start_point)
            time.sleep(1)

        blueprint_library = world.get_blueprint_library()

        camera_bp = blueprint_library.find('sensor.camera.semantic_segmentation')
        camera_bp.set_attribute('fov', '90')
        camera_bp.set_attribute('image_size_x', f'{WIDTH}')
        camera_bp.set_attribute('image_size_y', f'{HEIGHT}')
        camera_init_trans = carla.Transform(carla.Location(z=CAMERA_POS_Z,x=CAMERA_POS_X))
        camera_sem = world.spawn_actor(camera_bp,camera_init_trans,attach_to=vehicle)
        camera_data = {'sem_image': np.zeros((HEIGHT,WIDTH,4))}
        camera_sem.listen(lambda image: sem_callback(image,camera_data))

        collision_data ={'collision': False}
        collision_bp = world.get_blueprint_library().find('sensor.other.collision')
        collision_sensor = world.spawn_actor(collision_bp,carla.Transform(), attach_to = vehicle)
        collision_sensor.listen(lambda event: collision_callback(event,collision_data))

        route = select_random_route(world, start_point,spawn_points)
        curr_wp = 5
        predicted_angle = 0

        lidar_bp = generate_lidar_bp(blueprint_library)
        lidar_transform = carla.Transform(carla.Location(x=0, z=2))
        lidar = world.spawn_actor(lidar_bp, lidar_transform, attach_to=vehicle)
        point_list = o3d.geometry.PointCloud()
        lidar.listen(lambda data: lidar_callback(data, point_list))
        spectator = world.get_spectator()
        # generate_traffic(world, client, num_vehicles=80, num_pedestrians=10)
        while curr_wp<len(route)-1 :
            world.tick()
            update_spectator(vehicle)

            draw_route(world, curr_wp, route,1)
            if cv2.waitKey(1) == ord('q'):
                quit = True
                exit_clean(world, vehicle)
                break

            sem_image = camera_data['sem_image']
            sem_image = cv2.resize(sem_image, (WIDTH,HEIGHT))

            if curr_wp >=len(route)-10 or collision_data['collision']:
                exit_clean(world, vehicle)
                break

            while curr_wp<len(route)-2 and vehicle.get_transform().location.distance(route[curr_wp][0].transform.location)<5:
                curr_wp +=1 #move to next wp if we are too close
            curr_wp, predicted_angle = get_proper_angle(vehicle,curr_wp,route)
            distant_angle = get_distant_angle(vehicle,curr_wp,route)

            v = vehicle.get_velocity()
            speed = round(3.6 * math.sqrt(v.x**2 + v.y**2 + v.z**2),0)

            estimated_throttle = maintain_speed(speed)

            steer_input = control_car(sem_image, distant_angle)
            if steer_input > 0.3:
                steer_input = 0.3
            if steer_input < -0.3:
                steer_input = -0.3
            if (steer_input > -0.001) and (steer_input < 0.001):
                steer_input = 0

            vehicle.apply_control(carla.VehicleControl(throttle=float(estimated_throttle), steer=steer_input))


            if len(pcd1) > 0:
                latest_points = pcd1[-1]  # Get the latest point cloud frame
                has_points_in_front, mask, front_points = filter_points(latest_points)
                if has_points_in_front:
                    print(f"Obstacle Detected, Steering Angle: {steer_input}, Speed: {speed}")
                    vehicle.apply_control(carla.VehicleControl(throttle=0.0, brake=1.0))
                else:
                    print(f"No Obstacle Detected, Steering Angle: {steer_input}, Speed: {speed}")

        if quit:
            break

finally:

    for sensor in world.get_actors().filter('*sensor*'):
        sensor.destroy()
    for actor in world.get_actors().filter('*vehicle*'):
        actor.destroy()

# Version 2

In [ ]:
from PIL import Image
import open3d as o3d
from torchvision import transforms
import torch.nn.functional as F
import torch.nn as nn
import torch
from matplotlib import pyplot as plt
from agents.navigation.global_route_planner import GlobalRoutePlanner
import carla  # the sim library itself
import time  # to set a delay after each photo
import cv2  # to work with images from cameras
import numpy as np  # in this example to change image representation - re-shaping
import math
import sys
import random
# tweak to where you put carla
sys.path.append(
    r'C:\Users\Administrator\Downloads\WindowsNoEditor\PythonAPI\carla')


PREFERRED_SPEED = 20
SPEED_THRESHOLD = 2

MAX_STEER_DEGREES = 30
STEERING_CONVERSION = 75

CAMERA_POS_Z = 1.3
CAMERA_POS_X = 1.4

HEIGHT = 180
WIDTH = 320
X_MIN = -6
X_MAX = -1
Y_MIN = -1
Y_MAX = 1
Z_MIN = -1
Z_MAX = 2

WIDTH = 640
HEIGHT = 480

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")


def update_spectator(vehicle):
    vehicle_transform = vehicle.get_transform()

    # Position the spectator slightly behind and above the vehicle
    # Adjust x and z values as needed for view angle
    spectator_location = vehicle_transform.location + carla.Location(x=0, z=40)
    spectator_rotation = carla.Rotation(
        pitch=-90, yaw=vehicle_transform.rotation.yaw)  # Adjust pitch for angle

    spectator_transform = carla.Transform(
        spectator_location, spectator_rotation)
    spectator.set_transform(spectator_transform)


pcd1 = []


def lidar_callback(point_cloud, point_list):
    """Prepares a point cloud with intensity
    colors ready to be consumed by Open3D"""
    data = np.copy(np.frombuffer(point_cloud.raw_data, dtype=np.dtype('f4')))
    data = np.reshape(data, (int(data.shape[0] / 4), 4))
    points = data[:, :-1]
    points[:, :1] = -points[:, :1]

    pcd1.append(points)


def generate_lidar_bp(blueprint_library):
    """Generates a CARLA blue
    based on the script parameters"""
    lidar_bp = blueprint_library.find('sensor.lidar.ray_cast')

    lidar_bp.set_attribute('upper_fov', str(10.0))
    lidar_bp.set_attribute('lower_fov', str(-30.0))
    lidar_bp.set_attribute('channels', str(64.0))
    lidar_bp.set_attribute('range', str(100.0))
    lidar_bp.set_attribute('rotation_frequency', str(20))
    lidar_bp.set_attribute('points_per_second', str(1000000))
    return lidar_bp


def filter_points(points, x_min=X_MIN, x_max=X_MAX, y_min=Y_MIN, y_max=Y_MAX, z_min=Z_MIN, z_max=Z_MAX):
    mask = (
        (points[:, 0] >= x_min) & (points[:, 0] <= x_max) &  # x-axis (front)
        (points[:, 1] >= y_min) & (points[:, 1] <= y_max) &  # y-axis (lateral)
        (points[:, 2] >= z_min) & (points[:, 2] <= z_max)    # z-axis (height)
    )
    # Return if any point is in front and the points themselves
    return np.any(mask), mask, points[mask]


def camera_callback(image, data_dict):
    data_dict['image'] = np.reshape(
        np.copy(image.raw_data), (image.height, image.width, 4))[:, :, :3]


def sem_callback(image, data_dict):
    image.convert(carla.ColorConverter.CityScapesPalette)
    data_dict['sem_image'] = np.reshape(
        np.copy(image.raw_data), (image.height, image.width, 4))[:, :, :3]


def maintain_speed(s):
    if s >= PREFERRED_SPEED:
        return 0
    elif s < PREFERRED_SPEED - SPEED_THRESHOLD:
        return 0.7  # think of it as % of "full gas"
    else:
        return 0.4  # tweak this if the car is way over or under preferred speed


# function to get angle between the car and target waypoint
def get_angle(car, wp):
    vehicle_pos = car.get_transform()
    car_x = vehicle_pos.location.x
    car_y = vehicle_pos.location.y
    wp_x = wp.transform.location.x
    wp_y = wp.transform.location.y

    # vector to waypoint
    x = (wp_x - car_x)/((wp_y - car_y)**2 + (wp_x - car_x)**2)**0.5
    y = (wp_y - car_y)/((wp_y - car_y)**2 + (wp_x - car_x)**2)**0.5

    # car vector
    car_vector = vehicle_pos.get_forward_vector()
    degrees = math.degrees(np.arctan2(
        y, x) - np.arctan2(car_vector.y, car_vector.x))
    # extra checks on predicted angle when values close to 360 degrees are returned
    if degrees < -180:
        degrees = degrees + 360
    elif degrees > 180:
        degrees = degrees - 360
    return degrees


def get_proper_angle(car, wp_idx, rte):
    # create a list of angles to next 5 waypoints starting with current
    next_angle_list = []
    for i in range(5):
        if wp_idx + i*3 < len(rte)-1:
            next_angle_list.append(get_angle(car, rte[wp_idx + i*3][0]))
    idx = 0
    while idx < len(next_angle_list)-2 and abs(next_angle_list[idx]) > 40:
        idx += 1
    return wp_idx+idx*3, next_angle_list[idx]


def get_distant_angle(car, wp_idx, rte, delta=10):
    if wp_idx + delta < len(rte)-1:
        i = wp_idx + delta
    else:
        i = len(rte)-1
    intersection_detected = False
    for x in range(i-wp_idx):
        if rte[wp_idx+x][0].is_junction:
            intersection_detected = True
    angle = get_angle(car, rte[i][0])
    if not intersection_detected:
        result = 1
    elif angle < -10:
        result = 0
    elif angle > 10:
        result = 2
    else:
        result = 1
    return result


def draw_route(world, wp, route, seconds=3.0):
    if len(route)-wp < 25:
        draw_colour = carla.Color(r=255, g=0, b=0)
    else:
        draw_colour = carla.Color(r=0, g=0, b=255)
    for i in range(10):
        if wp+i < len(route)-2:
            world.debug.draw_string(route[wp+i][0].transform.location, '^', draw_shadow=False,
                                    color=draw_colour, life_time=seconds,
                                    persistent_lines=True)
    return None


def select_random_route(world, position, locs):
    point_a = position.location  # we start at where the car is or last waypoint
    sampling_resolution = 1
    grp = GlobalRoutePlanner(world.get_map(), sampling_resolution)
    # now let' pick the longest possible route
    min_distance = 100
    result_route = None
    route_list = []
    for loc in locs:  # we start trying all spawn points
        # but we just exclude first at zero index
        cur_route = grp.trace_route(point_a, loc.location)
        if len(cur_route) > min_distance:
            route_list.append(cur_route)
    result_route = random.choice(route_list)
    return result_route


def exit_clean(world, vehicle):
    for sensor in world.get_actors().filter('*sensor*'):
        sensor.destroy()
    # vehicle.destroy
    for actor in world.get_actors().filter('*vehicle*'):
        actor.destroy()
    return None


def collision_callback(event, data_dict):
    data_dict['collision'] = True


def preprocessing_image(image):
    image = image.convert('L')
    image = image.point(lambda x: 255 if x >= 70 else 0, '1')
    width, height = image.size
    left = (width - 300) // 2
    top = (height - 250) // 2
    right = left + 300
    bottom = top + 250
    return image.crop((left, top, right, bottom))


class AutoNaviModel(nn.Module):
    def __init__(self):
        super(AutoNaviModel, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(64 * (HEIGHT // 8) * (WIDTH // 8), 128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, 4),
            nn.ReLU(),
            nn.Flatten()
        )
        self.output_layer = nn.Linear(5, 1)  # 4 from image + 1 from input_2

    def forward(self, image, navigational_direction):
        image_features = self.conv_layers(image)

        combined = torch.cat(
            (image_features, navigational_direction.unsqueeze(1)), dim=1)

        output = self.output_layer(combined)
        return output


def control_car(sem_image, navigational_direction):
    sem_image_pil = Image.fromarray(sem_image.astype('uint8'), 'RGB')

    transform1 = transforms.Compose([
        transforms.Resize((HEIGHT, WIDTH)),
        transforms.ToTensor(),
    ])

    sem_image = transform1(sem_image_pil).unsqueeze(0).to(DEVICE)
    navigational_direction = torch.tensor([navigational_direction])

    with torch.no_grad():
        auto_navi_output = auto_navi_model(sem_image, navigational_direction)

    return float(auto_navi_output)


auto_navi_model_path = r'models\auto_navi_model.pth'
auto_navi_model = torch.load(
    auto_navi_model_path, map_location=torch.device(DEVICE))
auto_navi_model.eval()

# main loop
client = carla.Client('localhost', 2000)
world = client.get_world()
settings = world.get_settings()
settings.synchronous_mode = True  # Enable synchronous mode
settings.fixed_delta_seconds = 0.05  # Set the simulation time step (20 Hz)
world.apply_settings(settings)

spawn_points = world.get_map().get_spawn_points()

vehicle_bp = world.get_blueprint_library().filter('*model3*')
quit = False


try:
    while True:
        print("-----------Start Iteration------------")

        vehi = 0
        # Create a traffic manager and set its parameters
        # You can specify a different port if needed
        traffic_manager = client.get_trafficmanager(8000)
        # Example setting: minimum distance to the leading vehicle
        traffic_manager.set_global_distance_to_leading_vehicle(2.0)
        traffic_manager.set_synchronous_mode(True)

        # Now integrate it with vehicle spawning
        for i in range(0, len(spawn_points), 6):
            start_point = spawn_points[i]
            vehicle_blueprints = world.get_blueprint_library().filter('vehicle.*')
            vehicle_bp = random.choice(vehicle_blueprints)
            vehicle_1 = world.try_spawn_actor(vehicle_bp, start_point)
            time.sleep(0.5)

            if vehicle_1:  # If the vehicle was successfully spawned
                # Example setting: vehicles stop at traffic lights
                traffic_manager.ignore_lights_percentage(vehicle_1, 0)
                traffic_manager.vehicle_percentage_speed_difference(
                    vehicle_1, -10)  # Reduce vehicle speed by 10%
                # Connect the vehicle with the traffic manager
                vehicle_1.set_autopilot(True, traffic_manager.get_port())
                vehi += 1
                # print(vehi)
            # else:
            #     print(f"Failed to spawn vehicle at index {i}.")

        # start_point = random.choice(spawn_points)
        vehicle_bp = world.get_blueprint_library().filter('*model3*')
        start_point = spawn_points[2]
        # start_point =  random.choice(spawn_points)
        vehicle = world.try_spawn_actor(vehicle_bp[0], start_point)
        while vehicle is None:
            # start_point =  random.choice(spawn_points)
            start_point = spawn_points[2]
            vehicle = world.try_spawn_actor(vehicle_bp[0], start_point)
            time.sleep(1)

        blueprint_library = world.get_blueprint_library()

        camera_bp = blueprint_library.find(
            'sensor.camera.semantic_segmentation')
        camera_bp.set_attribute('fov', '90')
        camera_bp.set_attribute('image_size_x', f'{WIDTH}')
        camera_bp.set_attribute('image_size_y', f'{HEIGHT}')
        camera_init_trans = carla.Transform(
            carla.Location(z=CAMERA_POS_Z, x=CAMERA_POS_X))
        camera_sem = world.spawn_actor(
            camera_bp, camera_init_trans, attach_to=vehicle)
        camera_data = {'sem_image': np.zeros((HEIGHT, WIDTH, 4))}
        camera_sem.listen(lambda image: sem_callback(image, camera_data))

        collision_data = {'collision': False}
        collision_bp = world.get_blueprint_library().find('sensor.other.collision')
        collision_sensor = world.spawn_actor(
            collision_bp, carla.Transform(), attach_to=vehicle)
        collision_sensor.listen(
            lambda event: collision_callback(event, collision_data))

        route = select_random_route(world, start_point, spawn_points)
        curr_wp = 5
        predicted_angle = 0

        lidar_bp = generate_lidar_bp(blueprint_library)
        lidar_transform = carla.Transform(carla.Location(x=0, z=2))
        lidar = world.spawn_actor(lidar_bp, lidar_transform, attach_to=vehicle)
        point_list = o3d.geometry.PointCloud()
        lidar.listen(lambda data: lidar_callback(data, point_list))
        spectator = world.get_spectator()
        # generate_traffic(world, client, num_vehicles=80, num_pedestrians=10)
        while curr_wp < len(route)-1:
            world.tick()
            update_spectator(vehicle)

            draw_route(world, curr_wp, route, 1)
            if cv2.waitKey(1) == ord('q'):
                quit = True
                exit_clean(world, vehicle)
                break

            sem_image = camera_data['sem_image']
            sem_image = cv2.resize(sem_image, (WIDTH, HEIGHT))

            if curr_wp >= len(route)-10 or collision_data['collision']:
                exit_clean(world, vehicle)
                break

            while curr_wp < len(route)-2 and vehicle.get_transform().location.distance(route[curr_wp][0].transform.location) < 5:
                curr_wp += 1  # move to next wp if we are too close
            curr_wp, predicted_angle = get_proper_angle(
                vehicle, curr_wp, route)
            distant_angle = get_distant_angle(vehicle, curr_wp, route)

            v = vehicle.get_velocity()
            speed = round(3.6 * math.sqrt(v.x**2 + v.y**2 + v.z**2), 0)

            estimated_throttle = maintain_speed(speed)

            steer_input = control_car(sem_image, distant_angle)
            if steer_input > 0.3:
                steer_input = 0.3
            if steer_input < -0.3:
                steer_input = -0.3
            if (steer_input > -0.001) and (steer_input < 0.001):
                steer_input = 0

            vehicle.apply_control(carla.VehicleControl(
                throttle=float(estimated_throttle), steer=steer_input))

            if len(pcd1) > 0:
                latest_points = pcd1[-1]  # Get the latest point cloud frame
                has_points_in_front, mask, front_points = filter_points(
                    latest_points)
                if has_points_in_front:
                    print(f"Obstacle Detected, Steering Angle: {
                          steer_input}, Speed: {speed}")
                    vehicle.apply_control(
                        carla.VehicleControl(throttle=0.0, brake=1.0))
                else:
                    print(f"No Obstacle Detected, Steering Angle: {
                          steer_input}, Speed: {speed}")

        if quit:
            break

finally:

    for sensor in world.get_actors().filter('*sensor*'):
        sensor.destroy()
    for actor in world.get_actors().filter('*vehicle*'):
        actor.destroy()